In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options 
import pandas as pd
import bs4
import requests
import re
import numpy as np
import time

In [2]:
chrome_path=r"C:\Users\rongshiu\Desktop\chromedriver"
driver=webdriver.Chrome(chrome_path)

In [3]:
records=[]
for i in range(1,21):
    driver.get('https://sg.kompass.com/s/business-services/11/page-{}'.format(i))
    details_containers=driver.find_elements_by_css_selector('div.kompassImage')
    for details_container in details_containers:
        ActionChains(driver) \
        .key_down(Keys.CONTROL) \
        .click(details_container) \
        .key_up(Keys.CONTROL) \
        .perform()
        current_window = driver.current_window_handle
        new_window = [window for window in driver.window_handles if window != current_window][0]
        driver.switch_to.window(new_window)
        
        time.sleep(3)

        html=driver.page_source
        soup=bs4.BeautifulSoup(html,'lxml')
        
        if soup.find('div',class_='kompassImage ') !=None:
            image_url=soup.find('div',class_='kompassImage ')
            logo= [img['src'] for img in image_url.select('img[src]')][0]
        else:
            logo=""
            
            
        if soup.find('div',class_='headerRowLeft col-xs-12 col-sm-12 col-md-8') != None:
            company_name_container=soup.find('div',class_='headerRowLeft col-xs-12 col-sm-12 col-md-8')  
            company_name=company_name_container.find('h1').text.strip()
        else:
            company_name=""
        
        if soup.find('div', class_='addressCoordinates') !=None:
            address_container=soup.find('div', class_='addressCoordinates')
            address=address_container.find('p').text.strip()
            address=" ".join(address.split())
        else:
            address=""
        
        if soup.find('span', attrs={'itemprop':'description'}) !=None:
            presentation=soup.find('span', attrs={'itemprop':'description'}).text.strip()
        else:
            presentation=""
        
        general_info_list=[]
        if soup.find('div',class_='presentation global')!=None:
            general_info=soup.find('div',class_='presentation global')
            general=general_info.find_all('td')
            for info in general:
                general_info_list.append(info.text.strip())
            general_info_list=[e.replace('\xa0',' ') for e in general_info_list]
            list1,list2=general_info_list[::2],general_info_list[1::2]
            general_info_list=list(zip(list1,list2))
            if general_info.find('td',class_='listWww')!=None:
                website=general_info.find('td',class_='listWww').text.strip()
            else:
                website=""
                
        if len(soup.find_all('div',attrs={'class':'presentation'}))>=3:
            banks_container=soup.find_all('div',attrs={'class':'presentation'})[2]
            if banks_container.find('strong') !=None:
                banks=banks_container.find('strong').text.strip()
            else:
                banks=""
        else:
            banks=""
        
        if soup.find('p',class_='number') !=None:
            no_employees=soup.find_all('p',class_='number')[1].text.strip()
        else:
            no_employees=""
            
        link=driver.current_url
        
        brands_list=[]
        if soup.find('div',class_='presentation brand lhEqual tradenames') !=None:
            brands_container=soup.find('div',class_='presentation brand lhEqual tradenames')
            brands=brands_container.find_all('p',class_='tradeName')
            for brand in brands:
                brands_list.append(brand.text.strip())
                
        records.append((company_name,logo,address,presentation,general_info_list,website,banks,no_employees,brands_list,link))
        
        driver.close()
        driver.switch_to.window(current_window)

In [4]:
df1=pd.DataFrame(records,columns=['Company Name','Logo Image','Address','Presentation','General Information List','Website','Banks','No of Employees','Brands List','Link'])

In [5]:
df1.to_csv('Kompass_Crawler.csv',encoding='utf-8-sig',index=False)